In [39]:
#Technical
import os
import glob
import zipfile
import pandas as pd
import re
from datetime import datetime
import time
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

zipname = "MD.zip"

os.getcwd()

'/Users/aleksandrzybin/Documents/Study/ICEF_MSc2/Big Data and Machine Learning with Applications to Economics and Finance 2021-22/project'

In [25]:
#Get unique currency pairs from filenames
with zipfile.ZipFile(zipname, 'r') as f:
    file_names = f.namelist()
    
pair_names = set([item.split("_")[1] for item in file_names])
pair_names = [x for x in pair_names if "USDT" in x]
pair_names

['LINKUSDT',
 'DOTUSDT',
 'USDCUSDT',
 'TUSDUSDT',
 'XMRUSDT',
 'CVCUSDT',
 'ONTUSDT',
 'AAVEUSDT',
 'SHIBUSDT',
 'ALGOUSDT',
 'NEARUSDT',
 'IOTAUSDT',
 'ZECUSDT',
 'TRXUSDT',
 'BNBUSDT',
 'ICPUSDT',
 'NEOUSDT',
 'ONEUSDT',
 'STMXUSDT',
 'XRPUSDT',
 'DGBUSDT',
 'MKRUSDT',
 'DASHUSDT',
 'ADAUSDT',
 'BATUSDT',
 'UNIUSDT',
 'PAXUSDT',
 'AXSUSDT',
 'XLMUSDT',
 'LUNAUSDT',
 'AVAXUSDT',
 'XTZUSDT',
 'RVNUSDT',
 'QTUMUSDT',
 'SCUSDT',
 'STXUSDT',
 'LTCUSDT',
 'LRCUSDT',
 'DOGEUSDT',
 'ETHUSDT',
 'EOSUSDT',
 'COMPUSDT',
 'MATICUSDT',
 'FILUSDT',
 'BTCUSDT',
 'CELRUSDT',
 'ICXUSDT',
 'VETUSDT',
 'ZILUSDT',
 'SOLUSDT',
 'ETCUSDT',
 'BTTUSDT']

In [26]:
#Read csv files and merge them into a df based on the type of currency. Store dfs in a list.
zip_file = zipfile.ZipFile(zipname)
market_data = []
for name in pair_names:
    currency_pair = []
    for info in zip_file.infolist():
        if name in info.filename:
            print(name)
            df =  pd.read_csv(zip_file.open(info.filename), skiprows = 1)
            currency_pair.append(df)
    
    market_data_one_pair = pd.concat(currency_pair, axis=0, ignore_index=True)
    market_data.append(market_data_one_pair)
market_data_dictionary = dict(zip(pair_names, market_data))

LINKUSDT
LINKUSDT
LINKUSDT
DOTUSDT
USDCUSDT
USDCUSDT
USDCUSDT
TUSDUSDT
TUSDUSDT
TUSDUSDT
XMRUSDT
XMRUSDT
XMRUSDT
CVCUSDT
ONTUSDT
AAVEUSDT
SHIBUSDT
ALGOUSDT
NEARUSDT
IOTAUSDT
ZECUSDT
ZECUSDT
ZECUSDT
TRXUSDT
TRXUSDT
TRXUSDT
BNBUSDT
BNBUSDT
BNBUSDT
ICPUSDT
NEOUSDT
NEOUSDT
NEOUSDT
ONEUSDT
ONEUSDT
ONEUSDT
STMXUSDT
XRPUSDT
XRPUSDT
XRPUSDT
DGBUSDT
MKRUSDT
DASHUSDT
DASHUSDT
DASHUSDT
ADAUSDT
ADAUSDT
ADAUSDT
BATUSDT
BATUSDT
BATUSDT
UNIUSDT
PAXUSDT
PAXUSDT
AXSUSDT
XLMUSDT
XLMUSDT
XLMUSDT
LUNAUSDT
AVAXUSDT
XTZUSDT
RVNUSDT
QTUMUSDT
QTUMUSDT
QTUMUSDT
SCUSDT
STXUSDT
LTCUSDT
LTCUSDT
LTCUSDT
LRCUSDT
DOGEUSDT
DOGEUSDT
DOGEUSDT
DOGEUSDT
ETHUSDT
ETHUSDT
ETHUSDT
ETHUSDT
EOSUSDT
EOSUSDT
EOSUSDT
COMPUSDT
MATICUSDT
MATICUSDT
MATICUSDT
FILUSDT
BTCUSDT
BTCUSDT
BTCUSDT
BTCUSDT
CELRUSDT
CELRUSDT
CELRUSDT
ICXUSDT
VETUSDT
ZILUSDT
SOLUSDT
ETCUSDT
ETCUSDT
ETCUSDT
BTTUSDT
BTTUSDT
BTTUSDT


In [27]:
for key, value in market_data_dictionary.items():
    value.sort_values(by=['unix'], inplace=True)
    value['year'] = value['unix'].apply(lambda x: int(datetime.utcfromtimestamp(x/1000).strftime('%Y')))

In [33]:
names = []
min_time_values = []
max_time_values = []
observations_num_values = []
for key, value in market_data_dictionary.items():
    #get names
    names.append(key)
    #get min/max time
    min_time = datetime.utcfromtimestamp(min(value['unix'])/1000)
    max_time = datetime.utcfromtimestamp(max(value['unix'])/1000) #.strftime('%Y-%m-%d %H:%M:%S') - convert to string date
    min_time_values.append(min_time)
    max_time_values.append(max_time)
    #get number of observations
    observations_num = " ".join([str(round(len(value.index)/10**6, 3)), "mln"])
    observations_num_values.append(observations_num)
general_description = pd.DataFrame({"Currency Pair": names, 
                                   "Starting Date": min_time_values, "Ending Date": max_time_values, 
                                   "Number of obs.": observations_num_values})
general_description

,Currency Pair,Starting Date,Ending Date,Number of obs.
0,LINKUSDT,2020-01-17 08:00:00,2022-10-02 09:47:00,1.418 mln
1,DOTUSDT,2022-01-01 00:01:00,2022-10-02 09:47:00,0.395 mln
2,USDCUSDT,2020-09-11 20:41:00,2022-09-26 02:59:00,1.066 mln
3,TUSDUSDT,2020-09-11 20:41:00,2022-09-26 02:59:00,1.066 mln
4,XMRUSDT,2020-02-03 08:03:00,2022-10-02 08:47:00,1.394 mln
5,CVCUSDT,2022-01-01 00:01:00,2022-10-02 09:47:00,0.395 mln
6,ONTUSDT,2022-01-01 00:01:00,2022-10-02 09:48:00,0.395 mln
7,AAVEUSDT,2022-01-01 00:01:00,2022-10-02 09:47:00,0.395 mln
8,SHIBUSDT,2022-04-17 13:14:00,2022-10-02 10:47:00,0.241 mln
9,ALGOUSDT,2022-01-26 11:00:00,2022-10-02 09:48:00,0.358 mln


In [34]:
df_name = "Binance_BTCUSDT_d.csv"
df =  pd.read_csv(df_name, skiprows = 1)
df['date'] = df['date'].astype('datetime64[ns]')
df = df.set_index('date')

In [44]:
#https://plotly.com/python/time-series/#time-series-using-axes-of-type-date
fig = px.line(df, x=df.index, y="close", title="Daily Closing Price of BTCUSDT", 
             labels = {"close": "Close Price (USDT)",
                        "date": "Date"})
fig.show()

In [45]:
df['return'] = df['open'].pct_change()
fig = px.area(df, x=df.index, y=df['return'], title="Daily Return of BTCUSDT", 
             labels = {"return": "Return",
                        "date": "Date"})
fig.show()

In [51]:
df['volatility_30d'] = df['return'].rolling(30).std()*(365**0.5)
df['volatility_90d'] = df['return'].rolling(90).std()*(365**0.5)
df['volatility_300d'] = df['return'].rolling(300).std()*(365**0.5)

In [60]:
fig = px.line(df, x=df.index, y="volatility_30d", title="30-day rolling volatility of BTCUSDT", 
             labels = {"volatility_30d": "Volatility (30D)",
                        "date": "Date"}, width=800, height=800)
fig.show()

In [61]:
fig = px.line(df, x=df.index, y="volatility_90d", title="90-day rolling volatility of BTCUSDT", 
             labels = {"volatility_90d": "Volatility (90D)",
                        "date": "Date"}, width=800, height=800)
fig.show()

In [62]:
fig = px.line(df, x=df.index, y="volatility_300d", title="300-day rolling volatility of BTCUSDT", 
             labels = {"volatility_300d": "Volatility (300D)",
                        "date": "Date"}, width=800, height=800)
fig.show()